In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

### 데이터 전처리

In [3]:
# 날짜값 변경
def datePreprocessing(text):
    return text[0:6]
df_train['date'] = df_train['date'].apply(datePreprocessing)

# 제대로 됬나 확인
df_train['date'].unique()

array(['201410', '201502', '201406', '201501', '201504', '201405',
       '201503', '201407', '201412', '201408', '201411', '201409',
       '201505'], dtype=object)

In [4]:
df_train.shape

(15035, 21)

In [5]:
X = normalize(pd.concat([df_train.iloc[:,[1]], df_train.iloc[:,3:21]], axis=1))
y = df_train.iloc[:,[2]]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(10524, 19)
(4511, 19)
(10524, 1)
(4511, 1)


In [6]:
X_ph = tf.placeholder(tf.float32, shape=(None,X_train.shape[1]))
y_ph = tf.placeholder(tf.float32, shape=(None,1))
# hidden 1
w1 = tf.Variable(tf.random_normal([X_train.shape[1],15]))
b1 = tf.Variable(tf.zeros([15]))
hidden1 = tf.matmul(X_ph, w1) + b1

# hidden 2
w2 = tf.Variable(tf.random_normal([15,10]))
b2 = tf.Variable(tf.zeros([10]))
hidden2 = tf.matmul(hidden1, w2) + b2

# hidden 3
w3 = tf.Variable(tf.random_normal([10,5]))
b3 = tf.Variable(tf.zeros([5]))
hidden3 = tf.matmul(hidden2, w3) + b3

# output layer
W = tf.Variable(tf.random_normal([5,1]))
b = tf.Variable(tf.zeros([1]))
y_model = tf.matmul(hidden3, W) + b

In [7]:
train_size = X_train.shape[0]
learn_rate = 0.05
batch_size = 30
epoch = 20000

loss = tf.reduce_mean(tf.square(y_model - y_ph))
optimizer = tf.train.AdamOptimizer(learning_rate = learn_rate)
train = optimizer.minimize(loss)
init = tf.global_variables_initializer()

In [8]:
with tf.Session() as sess:
    sess.run(init)
    for i in range(epoch):
        rand_idx = np.random.randint(0, train_size, batch_size)
        rand_feed = {X_ph:X_train[rand_idx,:], y_ph:y_train.values[rand_idx,:]}
        sess.run(train,feed_dict=rand_feed)
        
        train_feed = {X_ph:X_train, y_ph:y_train}
        rmse = np.sqrt(sess.run(loss, feed_dict=train_feed))
        if (i+1) % 5000 == 0 : print(f'epoch = {i + 1}        rmse = {rmse}')
    train_feed = {X_ph:X_train, y_ph:y_train}
    test_feed = {X_ph:X_test, y_ph:y_test}
    
    inmse  = sess.run(loss, feed_dict=train_feed)
    outmse = sess.run(loss, feed_dict=test_feed)
        
    print(f'insample MSE = {inmse}')
    print(f'insample RMSE = {np.sqrt(inmse)}')
    print(f'outsample MSE = {outmse}')
    print(f'outsample RMSE = {np.sqrt(outmse)}')


epoch = 5000        rmse = 259607.96875
epoch = 10000        rmse = 253216.125
epoch = 15000        rmse = 255736.5
epoch = 20000        rmse = 254293.46875
insample MSE = 64665165824.0
insample RMSE = 254293.46875
outsample MSE = 74468425728.0
outsample RMSE = 272889.03125
